# Pseudo-node 20 in parallel mode for Gauss-fitting

In [1]:
import numpy as np
import scipy as sp
import time
import os
import shutil
from skimage.io import imread
import pandas as pd
from skimage import img_as_float
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import stats

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import precision_score, make_scorer, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

%matplotlib inline

# 0.1 Set of functions to fit an image with a gaussian

In [2]:
def gaus_moments(data):
    """Returns a dict containing initial guesses for 2D-Gaussian parameters: 
    (background, amplitude, x_mean, y_mean, sigma_x, sigma_y)
    
    Assuming that image is approximately centered"""
    total = data.sum()
    Y, X = np.indices(data.shape)
    x = data.shape[0]/2 #(X*data).sum()/total
    y = data.shape[1]/2 #(Y*data).sum()/total
    col = data[:, int(y)]
    sigma_y = np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum()
    row = data[int(x), :]
    sigma_x = np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum()
    height = data.min()
    amplitude = data.max() - height
    mom={'background':height, 'amplitude':amplitude, 'x_mean':x, 'y_mean':y, 'sigma_x':sigma_x, 'sigma_y':sigma_y}
    return mom

In [3]:
def twod_gaussian(height, amplitude, center_x, center_y, sig_x, sig_y, sig_xy):
    """Returns a gaussian function with the given parameters"""
    #Sigma = [[sig_x, sig_xy], [sig_xy, sig_y]]
    Sigmin= [[sig_y, -sig_xy], [-sig_xy, sig_x]]/(sig_x*sig_y - sig_xy**2 + 10**-4)
    center = np.array([center_x, center_y]).reshape((1,1,2))
    #print center.shape, Sigmin.shape, center.dot(Sigmin).shape
    return lambda x: height+ amplitude*np.exp(-( (x-center).dot(Sigmin)*(x-center) ).sum(axis=-1)/2)

In [4]:
def fit_gaussian(data):
    """Returns 2D-Gaussian fit parameters:
    (height, amplitude, center_x, center_y, sig_x, sig_y, sig_xy)
    found by a optimizing least squares of an error"""
    mome = gaus_moments(data)
    para = [mome['background'], mome['amplitude'], mome['x_mean'], mome['y_mean'], mome['sigma_x'], mome['sigma_y'], 1.2]
    x = np.stack(np.meshgrid(np.arange(data.shape[0]), np.arange(data.shape[1])), axis=-1)
    errorfunction = lambda p: np.ravel(twod_gaussian(*p)(x) - data)
    p, success = sp.optimize.leastsq(errorfunction, para)
    #print success
    return p

## 0.2 Set of functions for processing the data

In [5]:
def load_imgs(dir_name='dust_fog', numb = 10):
    """
    Loading 'numb' of images from 'dir_name'
    and reshaping each one into 8 polarization images of shape 30x30
    -------
    Output:   numpy array of shape 'numb' x 8 x 30 x 30
    """
    dir_name = 'D:/NEWSdm/crops/'+dir_name
    img_arr = []
    num_i = 0
    for img_name in os.listdir(dir_name):
        num_i += 1
        if num_i > numb :
            break
        tmp_im = (pd.read_csv(os.path.join(dir_name, img_name), header=None).drop(30, axis=1)).values.reshape((8,30,30))
        img_arr.append(tmp_im)
    return np.array(img_arr)

In [6]:
def imgs_to_params(imgs):
    """
    Takes array (shape=[numb,8,30,30]) of numb x 8 images of shape 30 x 30
    and fits a 2D Gaussian on each of the images.
    -------
    Output: DataFrame with fitting parameters for images in the array.
    """
    params = pd.DataFrame(columns=['img_num', 'polar', 'height', 'amplitude', 'center_x', 'center_y', 'sig_x', 'sig_y', 'sig_xy'])
    for num in xrange(imgs.shape[0]):
        i=0
        for data in imgs[num]:
            par = np.concatenate(([num+1, i], fit_gaussian(data)))
            par = par.reshape((1,len(par)))
            i += 1
            params = params.append(pd.DataFrame(par,columns=['img_num', 'polar', 'height', 'amplitude', 'center_x', 'center_y', 'sig_x', 'sig_y', 'sig_xy']), ignore_index=True)
    return params

In [7]:
def clean_params(params, up_bound=500):
    """
    Dropping divergent samples (rows) from the dataset.
    Sample is divergent if any parameter (except 'sig_xy') is negative or larger than 'up_bound'
    """
    img_num = params.get('img_num')
    cov_xy = params.get('sig_xy')
    params = params.drop(['img_num','sig_xy'],axis=1)
    params = params[params[params<up_bound]>=0]
    params = pd.concat([img_num, params, cov_xy], axis=1, join='inner')
    return params.dropna()

In [8]:
def params_to_features(params):
    """
    Getting physical features from fitting parameters:
    polarization number, amplitude, center position, rotation (represented by 'sig_xy'), minor axis, eccentrisity, area
    -------
    Output: DataFrame with features for every image
    """
    Sigms = params.get(['sig_x','sig_y','sig_xy']).values
    idx = params.index
    #print Sigms.shape
    a_ar = []
    area_ar = []
    eps_ar = []
    for s in Sigms:
        #print s
        a, b = np.linalg.eigvalsh([[s[0],s[2]],[s[2],s[1]]])
        if a>b:
            tmp = a
            a = b
            b = tmp
        a_ar.append(a)
        area_ar.append(np.pi * a * b)
        eps = b/a
        if eps<1: eps = 1/eps
        eps_ar.append(eps)
    ellip_par = pd.DataFrame(data={'minor_ax':a_ar, 'eps':eps_ar, 'area':area_ar}, index=idx)
    #print ellip_par.shape
    feat = params.drop(['img_num','height','sig_x','sig_y'],axis=1)
    feat = pd.concat([feat, ellip_par], axis=1)
    return feat

In [9]:
def feat_to_input(ft_data, cl_names):
    """
    Stacking data with names from 'cl_names' into one DataFrame labeling 'signal' with 1 and 'noise' with 0.
    """
    inp_data = pd.DataFrame(columns=np.concatenate((['target'], ft_data[cl_names[0]].columns.values)) )
    for name in cl_names:
        idx = ft_data[name].index
        if name[0] == 'C' :
            targ = pd.DataFrame(data={'target':np.ones(len(idx))}, index=idx)
        else: targ = pd.DataFrame(data={'target': np.zeros(len(idx))}, index=idx)
        app_data = pd.concat([targ, ft_data[name]], axis=1)
        inp_data = inp_data.append(app_data, ignore_index=True)
    return inp_data    

### 0.2.1 Special functions for large data samples

In [10]:
# should be improved

def load_and_fit_batches(class_name='dust_fog', numb = 10000, node=1, n_cpu=1, n_0 = 1000, fit_all = False):
    """
    Loading 'numb' of samples in total from 'class_name' and fitting them with Gaussians and writing results files.
    1. Taking 'n_0' samples at a time and rewriting the same variables on each iteration.
    2. Pseudo-parallelised by running 'n_cpu' applications at once by hands with different 'node' numbers.
    -------
    Output:
    Written 'csv' files with fitting parameters in specific folders, returns 'nump' totall number of samples processed.
    The resulting files are like 'C:/Artem/NEWS/params/batches/50000/gamma/cpu3_n1_it1_params.csv'
    
    *Important*: 'numb' must be devidible by 'n_0', otherwise up to 'n_0' extra samples wil be processe.
    """
    #Getting the number of iterations on this node.
    i_m = np.int(float(numb)/(n_0 * n_cpu))
    over = np.ceil((float(numb) - i_m*n_0*n_cpu)/n_0)
    if node <= over:
        i_m +=1 # each node has it's own i_m
        
    
    dir_name = 'C:/Artem/NEWS/crops/'+class_name
    nump = 0
    img_names = os.listdir(dir_name)
    for i in range(i_m):
        #iterating over samples in the batch
        img_arr = []
        for img_name in img_names[(node+i-1)*n_0 : (node+i)*n_0]:
            tmp_im = (pd.read_csv(os.path.join(dir_name, img_name), header=None).drop(30, axis=1)).values.reshape((8,30,30))
            img_arr.append(tmp_im)
        img_arr = np.array(img_arr)
        #getting parametrs of samples in the batch and writing tham to file.
        params = pd.DataFrame(columns=['img_num', 'polar', 'height', 'amplitude', 'center_x', 'center_y', 'sig_x', 'sig_y', 'sig_xy'])
        for img_num in np.arange(img_arr.shape[0]):
            polar=1
            for data in img_arr[img_num]:
                par = np.concatenate(([img_num+1, polar], fit_gaussian(data)))
                par = par.reshape((1,len(par)))
                polar += 1
                nump += 1
                params = params.append(pd.DataFrame(par,columns=['img_num', 'polar', 'height', 'amplitude', 'center_x', 'center_y', 'sig_x', 'sig_y', 'sig_xy']), ignore_index=True)
        output_name = str(numb)+'/'+class_name+'/cpu'+str(n_cpu)+'_n'+str(node)+'_it'+str(i+1)+'_params'
        if fit_all: output_name = 'ALL/'+class_name+'/cpu'+str(n_cpu)+'_n'+str(node)+'_it'+str(i+1)+'_params'
        params.to_csv('C:/Artem/NEWS/params/batches/'+output_name+'.csv')
    '''
    Пофиксить что самая последняя итерация выбегает за пределы, если не делится нацело.
    '''
    return nump

In [11]:
def read_params_from_names(numb=10000, name='dust_fog', fit_all=False):
    """
    Reading and stacking DataFrames with parameters from the specified directory: 
    C:/Artem/NEWS/params/batches/'numb'/name
    """
    dir_name = 'C:/Artem/NEWS/params/batches/'+str(numb)+'/'+name
    if fit_all: dir_name = 'C:/Artem/NEWS/params/batches/ALL/'+name
    params = pd.DataFrame(columns=['img_num', 'polar', 'height', 'amplitude', 'center_x', 'center_y', 'sig_x', 'sig_y', 'sig_xy'])
    for img_name in os.listdir(dir_name):
        par = pd.read_csv(os.path.join(dir_name, img_name), index_col=0)
        params = params.append(par, ignore_index=True)
    return params

In [12]:
def load_params_to_feat(numb, cl_names, giant=False):
    """
    Reading parameters from files, cleaning them and transforming into features.
    -------
    Output: Dict with features DataFrame for every class name
    """
    dat_params = {}
    ft_data = {}
    for name in cl_names:
        if giant:
            dat_params[name] = read_params_from_names(numb, name)
        else: dat_params[name] = pd.read_csv('C:/Artem/NEWS/params/'+str(numb)+'_'+name+'_params.csv', index_col=0)
        ft_data[name] = params_to_features( clean_params(dat_params[name]) )
    return ft_data

In [13]:
def feat_to_input_split(ft_data, cl_names, test_size=0.1, shuffle=True):
    """
    Splitting data for train-test with or without shuffle.
    Stacking specified classes together considering target label (signal/background).
    -------
    Output: X_train, X_test, y_train, y_test
    """
    if shuffle:
        inp_data = feat_to_input(ft_data, cl_names)
        X = inp_data.drop(['target'], axis=1).values
        y = inp_data.get('target').values
        X_tr, X_tt, y_tr, y_tt = train_test_split(X, y, test_size=test_size)
    else:
        y_tr = pd.DataFrame().values
        y_tt = pd.DataFrame().values
        X_tr = pd.DataFrame(columns=ft_data[cl_names[0]].columns).values
        X_tt = pd.DataFrame(columns=ft_data[cl_names[0]].columns).values
        for name in cl_names:
            if name[0]=='C': 
                y = np.ones(ft_data[name].shape[0])
            else: y = np.zeros(ft_data[name].shape[0])
            X = ft_data[name].values
            tmp_X_tr, tmp_X_tt, tmp_y_tr, tmp_y_tt = train_test_split(X, y, test_size=test_size, shuffle=False)
            X_tr = np.vstack((X_tr, tmp_X_tr))
            X_tt = np.vstack((X_tt, tmp_X_tt))
            y_tr = np.append(y_tr, tmp_y_tr)
            y_tt = np.append(y_tt, tmp_y_tt)
            
    return X_tr, X_tt, y_tr, y_tt

### 0.2.2 Functions for all polarisations stacked together

In [14]:
def params_to_stacked_polar(params, n_pol=8):
    """
    
    """
    pol_par = pd.DataFrame()
    for i in range(params.shape[0]//n_pol):
        par_slice = params[(i*8):((i+1)*8)]
        par_slice = pd.DataFrame(par_slice.drop(['img_num','polar'],axis=1).values, index=par_slice.get('polar'), columns=par_slice.drop(['img_num','polar'],axis=1).columns)
        #print par_slice.head(2), '\n'
        flat_slice = pd.DataFrame(index=[i])
        for j in np.arange(1,n_pol+1):
            col_slice = np.append(flat_slice.columns.values, [opa+str(int(j)) for opa in par_slice.loc[j].index.values])
            #print col_slice
            flat_slice = pd.DataFrame(np.append(flat_slice.values, par_slice.loc[j].values).reshape((1,-1)), columns=col_slice, index=[i])
        pol_par = pd.concat([pol_par, flat_slice])
    return pol_par

In [15]:
def clean_stacked_params(params, up_bound=500, n_pol=8):
    """
    Dropping divergent samples (rows) from the dataset.
    Sample is divergent if any parameter (except 'sig_xy') is negative or larger than 'up_bound'
    """
    img_num = params.index
    for par in params.columns:
        if par[:6] != 'sig_xy':
            params = params[ params[par]>=0 ]
        params = params[ params[par]<up_bound ]
    print (params.shape)
    return params.dropna()

In [16]:
def params_to_feat_stacked(params, n_pol=8):
    """
    
    """
    idx = params.index
    feat = params.copy()
    for i in np.arange(1,n_pol+1):
        Sigms = params.get(['sig_x'+str(i),'sig_y'+str(i),'sig_xy'+str(i)]).values
        a_ar = []
        area_ar = []
        eps_ar = []
        b_ar = []
        for s in Sigms:
            a, b = np.linalg.eigvalsh([[s[0],s[2]],[s[2],s[1]]])
            if a>b:
                tmp = a
                a = b
                b = tmp
            a_ar.append(a)
            area_ar.append(np.pi * a * b)
            b_ar.append(b)
            eps_ar.append(b/a)
        ellip_par = pd.DataFrame(data={'min_ax'+str(i):a_ar, 'maj_ax'+str(i):b_ar, 'eps'+str(i):eps_ar, 'area'+str(i):area_ar}, index=idx)
        feat = feat.drop(['height'+str(i),'sig_x'+str(i),'sig_y'+str(i)],axis=1)
        feat = pd.concat([feat, ellip_par], axis=1)
    return feat

In [17]:
class_names = ['C60keV','C80keV','C100keV','dust_fog','gamma']

## 1. Fitting on ALL samples of each type

In [18]:
repeated = False

##### 1.1 Fitting the images and saving parameters into files
* _repeated_ is for recreating the output directory if you are running the same number of samples not for the first time
* 20000 images on two nodes ~14hours

In [20]:
%%time
numb = {}
numb['C60keV'] = 123000
numb['C80keV'] = 171000
numb['C100keV'] = 161000
numb['dust_fog'] = 173000
numb['gamma'] = 164000
giant_data_nump = {}
for name in class_names:
    giant_data_nump[name] = load_and_fit_batches(class_name=name, numb=numb[name], node=20, n_cpu=20, fit_all=True)
print (giant_data_nump)

C:\Artem\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
C:\Artem\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in multiply
  import sys
C:\Artem\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 1600.
  warnings.warn(errors[info][0], RuntimeWarning)


{'C60keV': 48000, 'C80keV': 64000, 'C100keV': 64000, 'dust_fog': 64000, 'gamma': 64000}
Wall time: 4h 20min 14s


# 51. TESTING AREA

##### <font color='red'> RANDOM CODE HAPPENS </font>